In [ ]:
import os
import pandas as pd
import zipfile
import geopandas as  gpd
from fiona.crs import from_epsg
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import seaborn
import datetime
import pylab as pl
%pylab inline

In [ ]:
def get_data():
    '''
    Download data for Homework 11, Assigment 2 and move to HW11 PUI Data Folder, including:
    1.  Census Data for Businesses by Zipcode
    2.  NYC Zipcodes Shapefile 
    '''

    # Download NYC Zipcode Geojson File in HW11 githib repository
    
    data_url = "https://raw.githubusercontent.com/fedhere/PUI2016_fb55/master/\
    HW11_fb55/nyc-zip-code-tabulation-areas-polygons.geojson"
    
    !curl -O "{data_url}"

    # Download Census Data for Business by Zip Code (Code provided by professor)

    !for ((y=93; y<=99; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done 
    !for ((y=0; y<=1; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done
    !for ((y=2; y<=9; y+=1)); do wget ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done
    !for ((y=10; y<=15; y+=1)); do wget ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done
    
get_data()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    21    0    21    0     0     67      0 --:--:-- --:--:-- --:--:--  1235
--2016-12-14 21:31:04--  ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp93totals.zip
           => “zbp93totals.zip”
Resolving ftp.census.gov... 148.129.75.35, 2610:20:2010:a09:1000:0:9481:4b23
Connecting to ftp.census.gov|148.129.75.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Econ2001_And_Earlier/CBP_CSV ... done.
==> SIZE zbp93totals.zip ... done.
==> PASV ... done.    ==> RETR zbp93totals.zip ... 
No such file “zbp93totals.zip”.

--2016-12-14 21:31:04--  ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp94totals.zip
           => “zbp94totals.zip.5”
Resolving ftp.census.gov... 148.129.75.35, 2610:20:2010:a09:1000:0:9481:4b23
Connecting to ftp.census.go

In [ ]:
import pandas as pd

import zipfiles #i am not giving it a name cause i intend to use it only once

zf = zipfile.ZipFile(fname)
df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))

In [ ]:
#puidata = os.getenv('PUIDATA')
#open zipcode file stored in PUIDATA
nycshape = gpd.read_file("nyc-zip-code-tabulation-areas-polygons.geojson")
nycshape.head()

In [ ]:
# rename postalcode column name to zipcode and remove columns deemed unnecessary
nycshape = nycshape.rename(columns={'postalCode': "zipcode"})
nycshape = nycshape[['zipcode', 'borough', 'geometry', 'PO_NAME']]
#datatype for all columns is 'object', changing zipcode to 'numeric'
nycshape.zipcode = pd.to_numeric(nycshape.zipcode)
nycshape.head()

In [ ]:
#create a new dataframe with values from all census business data files, 
years = ['94', '95', '96', '97', '98', '99', '00', '01', '02', '03', '04', '05', '06', '07',
        '08', '09', '10', '11', '12', '13', '14']
CombinedValues = pd.DataFrame()
for year in years:
    fname = 'zbp' + year + 'totals.zip'
    zf = zipfile.ZipFile(fname)
    each_year = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    each_year['year'] = year
    CombinedValues = pd.concat([CombinedValues, each_year], axis=0)
    
CombinedValues.head()

In [ ]:
CombinedValues_reduced = CombinedValues
CombinedValues_reduced.drop(CombinedValues_reduced.columns[[0,1,2,3,4,
                                                            5,6,7,8,9,10,11,12,13,15,
                                                            16,17,18]], axis=1, inplace=True)
CombinedValues_reduced.head()

In [ ]:
#convert zipcode to numeric and rename zip column to zipcode for the merge
CombinedValues_reduced["zip"] = CombinedValues_reduced["zip"].apply(pd.to_numeric)
CombinedValues_reduced = CombinedValues_reduced.rename(columns={'zip': 'zipcode'})

#merge shapefile and census information together on zipcode column
combinedtest = nycshape.merge(CombinedValues_reduced, on = 'zipcode')
combinedtest.head()